# Import Required Libraries

In [1]:
%pip install googletrans==4.0.0-rc1 --quiet
%pip install pandas --quiet
%pip install nltk --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Import necessary libraries
from googletrans import Translator
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re

# Load the CSV File

In [45]:
import pandas as pd

# Define the absolute path to the CSV file
csv_file_path = r'C:\Users\Jonah Dalton\Data_Pipeline\Twitter_Scrape\tweets.csv'

# Read the CSV file into a DataFrame
try:
    df = pd.read_csv(csv_file_path)
    # Display the DataFrame
    print(df.head(10))
except FileNotFoundError:
    print(f"File not found: {csv_file_path}")
except pd.errors.EmptyDataError:
    print("The file is empty.")
except Exception as e:
    print(f"An error occurred: {e}")

               Post ID                                       URL to tweet  \
0  1828837775791341997  https://x.com/JimmyMatthewsJr/status/182883777...   
1  1828607916892422511  https://x.com/JimmyMatthewsJr/status/182860791...   
2  1826979750856049026  https://x.com/TheVladsven/status/1826979750856...   
3  1822071851960692972  https://x.com/TheVladsven/status/1822071851960...   
4  1822069995565883579  https://x.com/TheVladsven/status/1822069995565...   
5  1828212669670629624  https://x.com/Bloomer99999/status/182821266967...   
6  1828060378711945505  https://x.com/EmilyMach5/status/18280603787119...   
7  1813345368484073626  https://x.com/ScottWIsAnIdiot/status/181334536...   
8  1829862469894058286  https://x.com/EmilyMach5/status/18298624698940...   
9  1823835478480605640  https://x.com/acnewsitics/status/1823835478480...   

                    creation_date  likes  \
0  Wed Aug 28 16:51:18 +0000 2024     29   
1  Wed Aug 28 01:37:55 +0000 2024     21   
2  Fri Aug 23 13:48:

In [46]:
# Extract and remove mentions
df['mentions'] = df['caption'].apply(lambda x: ' '.join(re.findall(r'@\w+', x)))
df['caption_no_mentions'] = df['caption'].apply(lambda x: re.sub(r'@\w+', '', x))

# Extract hashtags
df['new_hashtags'] = df['caption_no_mentions'].apply(lambda x: ' '.join(re.findall(r'#\w+', x)))
df['hashtags'] = df['hashtags'].mask(df['hashtags'].isna() | (df['hashtags'] == ''), df['new_hashtags'])
df.drop(columns=['new_hashtags'], inplace=True)

# Extract and remove URLs
df['urls'] = df['caption_no_mentions'].apply(lambda x: ' '.join(re.findall(r'http\S+', x)))
df['urls'] = df['urls'].str.replace(r'https://t\.co/\w+', '', regex=True)
df['caption_clean'] = df['caption_no_mentions'].apply(lambda x: re.sub(r'http\S+', '', x))

#remove a ton of extra white space from caption_clean
df['caption_clean'] = df['caption_clean'].str.replace(r'\s+', ' ', regex=True)
#drop filtering collumns
df.drop(columns=['caption_no_mentions'], inplace=True)
df.rename(columns={'caption': 'original_caption'}, inplace=True)
print(df.head(10))


               Post ID                                       URL to tweet  \
0  1828837775791341997  https://x.com/JimmyMatthewsJr/status/182883777...   
1  1828607916892422511  https://x.com/JimmyMatthewsJr/status/182860791...   
2  1826979750856049026  https://x.com/TheVladsven/status/1826979750856...   
3  1822071851960692972  https://x.com/TheVladsven/status/1822071851960...   
4  1822069995565883579  https://x.com/TheVladsven/status/1822069995565...   
5  1828212669670629624  https://x.com/Bloomer99999/status/182821266967...   
6  1828060378711945505  https://x.com/EmilyMach5/status/18280603787119...   
7  1813345368484073626  https://x.com/ScottWIsAnIdiot/status/181334536...   
8  1829862469894058286  https://x.com/EmilyMach5/status/18298624698940...   
9  1823835478480605640  https://x.com/acnewsitics/status/1823835478480...   

                    creation_date  likes  \
0  Wed Aug 28 16:51:18 +0000 2024     29   
1  Wed Aug 28 01:37:55 +0000 2024     21   
2  Fri Aug 23 13:48:

# Sentiment Score

In [47]:

nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Perform sentiment analysis on the 'Text in English language' column and save the results to a new column
def analyze_sentiment(text):
    sentiment_scores = sia.polarity_scores(text)
    return sentiment_scores['pos'], sentiment_scores['neu'], sentiment_scores['neg'], sentiment_scores['compound']

# Apply the analyze_sentiment function to the 'Text in English language' column
df['Positive'], df['Neutral'], df['Negative'], df['Compound'] = zip(*df['caption_clean'].apply(analyze_sentiment))

# Display the first few rows of the DataFrame to verify the sentiment analysis
print(df.head())


               Post ID                                       URL to tweet  \
0  1828837775791341997  https://x.com/JimmyMatthewsJr/status/182883777...   
1  1828607916892422511  https://x.com/JimmyMatthewsJr/status/182860791...   
2  1826979750856049026  https://x.com/TheVladsven/status/1826979750856...   
3  1822071851960692972  https://x.com/TheVladsven/status/1822071851960...   
4  1822069995565883579  https://x.com/TheVladsven/status/1822069995565...   

                    creation_date  likes  \
0  Wed Aug 28 16:51:18 +0000 2024     29   
1  Wed Aug 28 01:37:55 +0000 2024     21   
2  Fri Aug 23 13:48:10 +0000 2024      1   
3  Sat Aug 10 00:45:56 +0000 2024     19   
4  Sat Aug 10 00:38:34 +0000 2024      8   

                                    original_caption  retweet_count  \
0  #TrumpIsUnfitForOffice #ArlingtonNationalCemet...             18   
1  @thedesertginger I would not be surprised if h...              5   
2  Respect marriage.  Donald Trump and Where's Me...       

[nltk_data] Downloading package vader_lexicon to C:\Users\Jonah
[nltk_data]     Dalton\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


# Import Spacy Packages

In [20]:
%pip install -U pip setuptools wheel
%pip install -U spacy
!python -m spacy download en_core_web_sm

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: pip in c:\users\jonah dalton\data_pipeline\.venv\lib\site-packages (24.3.1)
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 1.8/1.8 MB 14.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 1.2/1.2 MB 8.9 MB/s eta 0:00:00
  Attempting uninstall: wheel
    Found existing installation: wheel 0.45.0
    Uninstalling wheel-0.45.0:
      Successfully uninstalled wheel-0.45.0
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.5.0
    Uninstalling setuptools-75.5.0:
      Successfully uninstalled setuptools-75.5.0
  Attempting uninstall: pip
    Found existing installation: pip 24.3.1
    Uninstalling pip-24.3.1:
      Successfully uninstalled pip-24.3.1



ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.2 requires numpy<2.1.0,>=2.0.0; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.


   ---------------------------------------- 0.0/11.8 MB ? eta -:--:--
   ------------- -------------------------- 3.9/11.8 MB 26.0 MB/s eta 0:00:01
   ----------------------------- ---------- 8.7/11.8 MB 23.3 MB/s eta 0:00:01
   ---------------------------------------- 11.8/11.8 MB 21.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 25.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   -------------------------------------- - 6.0/6.3 MB 30.8 MB/s eta 0:00:01
   ---------------------------------------- 6.3/6.3 MB 27.4 MB/s eta 0:00:00
  Attempting uninstall: blis
    Found existing installation: blis 1.0.1
    Uninstalling blis-1.0.1:
      Successfully uninstalled blis-1.0.1
  Attempting uninstall: thinc
    Found existing installation: thinc 8.3.2
    Uninstalling thinc-8.3.2:
      Successfully uninstalled thinc-8.3.2
  Attempting uninstall: spacy
    

# Extract Entities

In [48]:
import spacy

# Load the spaCy English model
nlp = spacy.load("en_core_web_sm")

# Function to perform NER and separate entities by type
def extract_entities_by_type(text):
    doc = nlp(text)
    persons = [entity.text for entity in doc.ents if entity.label_ == "PERSON"]
    orgs = [entity.text for entity in doc.ents if entity.label_ == "ORG"]
    gpes = [entity.text for entity in doc.ents if entity.label_ == "GPE"]
    rel = [entity.text for entity in doc.ents if entity.label_ == "NORP"]
    dat = [entity.text for entity in doc.ents if entity.label_ == "DATE"]
    return persons, orgs, gpes, rel, dat

# Apply the function to each tweet and store the results in separate columns
df[['Persons', 'Organizations', 'Locations', 'Nationalities/religiious groups', 'Date']] = df['caption_clean'].apply(
    lambda x: pd.Series(extract_entities_by_type(x))
)


# Topic Modeling

In [52]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import string
import re

# Preprocess text function (removes punctuation)
def preprocess_text(text):
    if isinstance(text, str):  # Ensure the text is a string
        text = text.lower()  # Convert text to lowercase
        text = ''.join([char for char in text if char not in string.punctuation])  # Remove punctuation
    return text

# Apply preprocessing
df['topic_model_clean'] = df['caption_clean'].apply(preprocess_text)

# **Replace TF-IDF with CountVectorizer**
vectorizer = CountVectorizer(stop_words='english', max_df=0.93, min_df=3)
X = vectorizer.fit_transform(df['topic_model_clean'])

# **Train LDA Model**
num_topics = 7
lda_model = LatentDirichletAllocation(n_components=num_topics, max_iter=10, learning_method='online', random_state=42)
lda_model.fit(X)

# Get top words for each topic
terms = vectorizer.get_feature_names_out()
n_top_words = 10
topic_keywords = {}

topics = {
    0: "Patriotism & Trump Support Network",
    1: "MAGA & Election Enthusiasm",
    2: "Voting & Opposition to Kamala Harris",
    3: "Voter Readiness & Motivation",
    4: "Canadites Foreign Loyalty",
    5: "Harris Criticism & Justice Themes",
    6: "Trump Loyalty & Enthusiasm",
    7: "Elections & Trump Presidency"
}

for topic_idx, topic in enumerate(lda_model.components_):
    top_words = [terms[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
    topic_keywords[topic_idx] = top_words
    print(f"Topic #{topic_idx} ({topics[topic_idx]}): {', '.join(top_words)}")
    print(f"Weights: {topic[topic.argsort()[:-n_top_words - 1:-1]]}")

# **Assign Topics to Each Document**
topic_assignments = lda_model.transform(X)
df['topic_id'] = topic_assignments.argmax(axis=1)

# **Map Topics to Labels**
df['topic_label'] = df['topic_id'].map(topics)

# Remove processed text column
df.drop('topic_model_clean', axis=1, inplace=True)

# Print the first 10 rows with assigned topics
print(df[['original_caption', 'topic_label']].head(10))


Topic #0 (Patriotism & Trump Support Network): remember, lets, trumpisunfitforoffice, maga, great, happy, war, think, support, fight
Weights: [3.5112127  2.32282135 2.24065639 2.1360738  1.54219845 1.52615314
 0.92880969 0.91828296 0.90672634 0.90497616]
Topic #1 (MAGA & Election Enthusiasm): got, ve, fuck, big, wait, november, babies, voting, votetrump2024tosaveamerica, proud
Weights: [1.58830027 1.58336951 1.53716691 0.92613291 0.9233155  0.91821055
 0.90222693 0.87026681 0.85767267 0.80575576]
Topic #2 (Voting & Opposition to Kamala Harris): patriots, vote, trumpvance2024tosaveamerica, maga, thanks, president, voted, follow, january, want
Weights: [6.7202935  4.71532963 3.31449109 3.04678504 2.95653657 2.94591133
 2.6740844  2.30380885 2.24764323 2.23219433]
Topic #3 (Voter Readiness & Motivation): maga, maga2024, trump2024, kamala, vote, don, trump2024vance, real, forget, repost
Weights: [10.96396811  8.46139658  5.33786982  5.21818483  3.31829418  2.99471261
  2.97477014  2.646186

# Save the New CSV File

In [50]:
# Select the required columns for the new CSV file

new_df = df[['caption_clean', 'image+caption_target', 'political_leaning', 'topic_label', 'mentions', 'hashtags',  'urls', 'likes', 'retweet_count', 'comment_count', 'view_count', 'poster_follower_count', 'poster_frequency', 'total_posts', 
    'image_filename', 'comment_file', 'Positive', 'Neutral', 'Negative', 'Compound', 'Persons', 'Organizations', 'Locations', 'Nationalities/religiious groups', 'Date',  'Post ID', 'URL to tweet', 'creation_date', 'poster_handle', 'poster_display_name',]]

# Save the new DataFrame to a CSV file
new_df.to_csv('Processed_Tweets.csv', index=False)